# TRAINING

In [5]:
import pandas as pd
import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
import numpy as np
from sklearn.svm import LinearSVC
def filter_label(df,label):
    return df[df['label']==label].reset_index()

filetrain='/root/Aril/CLUSTERING/20190116_data_train.csv'
df_all = pd.read_csv(file_train, sep='\t')
df_all.head(2)


,Unnamed: 0,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,...,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3.0,3.15,3.25,4.4,n,y,...,n,y,06/19/09 03:21 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
1,1,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3.0,3.15,3.25,4.4,n,y,...,n,y,07/02/09 08:41 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1


In [62]:
df=df_all.rename(columns={'page_name':'Teks','tendency total':'label'})[['Teks','label']]
df.label.value_counts()

swing      758602
jokowi     758594
prabowo    758592
Name: label, dtype: int64

In [63]:
j_train=filter_label(df,'jokowi')
p_train=filter_label(df,'prabowo')
s_train=filter_label(df,'swing')

In [64]:
df.label.value_counts()

swing      758602
jokowi     758594
prabowo    758592
Name: label, dtype: int64

In [72]:
df=j_train.append(p_train).sample(n=1500000)
df.label.value_counts()

jokowi     750101
prabowo    749899
Name: label, dtype: int64

In [73]:
X_train, X_test, y_train, y_test = train_test_split(df["Teks"], df["label"], test_size=0.2, random_state=123) 

models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]

models=[ LinearSVC(tol=0.00001)]
for model in models:
#     max_df  = 100  #semakin kecil semakin banyak yg dibuang (frek. terlalu tinggi)
#     min_df = 2 #semakin besar semakin banyak yg dibuang (freq. terlalu kecil)  
#     cvec = CountVectorizer(max_df=max_df, min_df=min_df)
    cvec = CountVectorizer()
    model_name = model.__class__.__name__
    # cvec = CountVectorizer()
    tfidf_trans = TfidfTransformer()
    text_clf = Pipeline([('vect', cvec),
                        ('tfidf',tfidf_trans),
                         ('clf', model)])
    text_clf.fit(X_train, y_train)
    pred = text_clf.predict(X_test)
    akurasi = np.mean(pred==y_test)
    print(model_name,"Akurasi: {}".format(akurasi))
    print(classification_report(y_test, pred))



LinearSVC Akurasi: 0.9544166666666667
             precision    recall  f1-score   support

     jokowi       0.95      0.96      0.95    149842
    prabowo       0.96      0.95      0.95    150158

avg / total       0.95      0.95      0.95    300000



In [ ]:
text_clf.fit(X_test, y_test)
pred = text_clf.predict(X_train)
akurasi = np.mean(pred==y_train)

# TES


In [78]:

import dask.dataframe as dd
df_tes = dd.read_csv('/root/Aril/CLUSTERING/data_train_aril.csv', sep='\t')
df_tes.head(2)
# df_tes=df_tes.rename(columns={'page_name':'Teks','tendency total':'label'})[['Teks','label']]

,user_profile_link,page_name,tendency total
0,https://www.facebook.com/100000127777885,Rock Scale RC Club kumparan Zilingo Nokia Mobi...,jokowi
1,https://www.facebook.com/100000187875203,Pengembang.Com Griya Cibinong Asri PT Megannes...,jokowi


In [81]:
df_tes=df_tes.rename(columns={'page_name':'Teks','tendency total':'label'})[['Teks','label']]
# len(df_tes)
j_tes=filter_label(df_tes,'jokowi')
p_tes=filter_label(df_tes,'prabowo')

In [100]:
c=j_tes.head(1000).append(p_tes.head(1000))

In [101]:
sms_baru = ['jokowi apaaja prabowo gerindra pdi jokowi lagi']
pred = text_clf.predict(sms_baru)
print("Hasil prediksi: {}".format(pred))

c['pred']=text_clf.predict(c.Teks)
(len(c)-len(c[c['label']!=c['pred']]))/len(c)

Hasil prediksi: ['jokowi']


0.993

In [ ]:
hasil1 = 0.9544166666666667
hasil2 = 0.9531538461538461

In [74]:
#save model
import pickle
filename = 'akurasi95%.sav'
pickle.dump(text_clf, open(filename, 'wb'))

In [4]:
#load model
import pickle
loaded_model = pickle.load(open(filename, 'rb'))
loaded_model

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=1e-05,
     verbose=0))])

# MACHINE LEARNING FOR PREDICT TENDENCY BY PAGE NAME

In [3]:
import pickle
model= pickle.load(open('/root/Aril/CLUSTERING/machine SMS/akurasi95%.sav','rb'))
model

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=1e-05,
     verbose=0))])

In [27]:
data=pd.read_csv('/root/Aril/CLUSTERING/data_237.csv')

In [24]:
data=data[:10]

In [28]:
data['predict']=model.predict(data['page_name'])

In [33]:
model.predict(["jokow"])

array(['jokowi'], dtype=object)

In [7]:
import argparse
import pandas as pd

In [6]:
import argparse
import pandas as pd
import pickle

if __name__ == "__main__":
    ap= argparse.ArgumentParser()
    ap.add_argument("-file", type=str,required=True)
    ap.add_argument("-column", type=str,required=True)
    ap.add_argument("-sep", type=str, default=',',required=False)
    args=ap.parse_args()
    file=args.file
    column=args.column
    sep=args.sep
    print('load model...')
    model= pickle.load(open('/root/Aril/CLUSTERING/machine SMS/akurasi95%.sav','rb'))
    print(f'file {file} will be predicted..')
    df=pd.read_csv(file, sep=sep, engine='python')
    df=df[:10]
    df['predict']=model.predict(df[column])
    print(df[['predict','label']])


usage: ipykernel_launcher.py [-h] -file FILE -column COLUMN
ipykernel_launcher.py: error: the following arguments are required: -column


SystemExit: 2

/root/opt/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [8]:
filetrain='/root/Aril/CLUSTERING/20190116_data_train.csv'

In [9]:
import re

In [14]:
filetrain.replace(re.findall(r'/\w+.csv',filetrain)[0],'')

'/root/Aril/CLUSTERING'

In [ ]:
load